In [1]:
from qbob import qbob, qpam
from qbob.intrinsics import *
from qbob.types import *

In [2]:
my_qbob = qbob.OperationBuilder("HelloWorld")
my_qbob.is_entrypoint = True
my_qbob += Message("Hello World!")
my_qbob += Message("Hello Again!")

In [3]:
print(my_qbob.build())

@EntryPoint()
operation HelloWorld () : Unit {
    Message("Hello World!");
    Message("Hello Again!");
}


In [4]:
my_qpam = qpam.ProgramArchitect("MyProject")
my_qpam.is_executable = True
my_qpam.add_operations(my_qbob)
my_qpam.create_project(".")

'MyProject.csproj'

In [5]:
!type Program.qs

namespace MyProject {
    open Microsoft.Quantum.Canon;
    open Microsoft.Quantum.Intrinsic;

    @EntryPoint()
    operation HelloWorld () : Unit {
        Message("Hello World!");
        Message("Hello Again!");
    }
}


In [10]:
!dotnet run

Hello World!
Hello Again!


In [7]:
my_qbob = qbob.OperationBuilder("Teleport")
msg = my_qbob.input("msg", Qubit)
target = my_qbob.input("target", Qubit)
with my_qbob.allocate_qubits("auxiliary", 1) as aux:
    my_qbob += H(aux)
    my_qbob += CNOT(aux, target)
    my_qbob += CNOT(msg, aux)
    my_qbob += H(msg)
    with my_qbob.if_statement(Measure([Pauli.PauliZ], [msg]) == One):
        my_qbob += Z(target)
    with my_qbob.if_statement(Measure([Pauli.PauliZ], [aux]) == One):
        my_qbob += X(target)
        
print(my_qbob.build())

operation Teleport (msg : Qubit, target : Qubit) : Unit {
    using (auxiliary = Qubit()) {
        H(auxiliary);
        CNOT(auxiliary, target);
        CNOT(msg, auxiliary);
        H(msg);

        if (Measure([PauliZ], [msg]) == One) {
            Z(target);
        }

        if (Measure([PauliZ], [auxiliary]) == One) {
            X(target);
        }
    }
}


In [8]:
prepare_qbob = qbob.OperationBuilder("PrepareEntangledState")
prepare_qbob.is_adj = True
qubits = prepare_qbob.input("qubits", List[Qubit])
prepare_qbob += H(qubits[0])
prepare_qbob += CNOT(qubits[0], qubits[1])

print(prepare_qbob.build())

measure_qbob = qbob.OperationBuilder("MeasureEntangledState")
with measure_qbob.allocate_qubits("qubits", 2) as q:
    measure_qbob += prepare_qbob(q)
    measure_qbob.returns([M(q[0]), M(q[1])])
    
print(measure_qbob.build())

operation PrepareEntangledState (qubits : Qubit[]) : Unit is Adj {
    H(qubits[0]);
    CNOT(qubits[0], qubits[1]);
}
operation MeasureEntangledState () : Result[] {
    using (qubits = Qubit[2]) {
        PrepareEntangledState(qubits);
        return [M(qubits[0]), M(qubits[1])];
    }
}


In [9]:
import qsharp
prepare_op = qsharp.compile(prepare_qbob.build())
measure_op = qsharp.compile(measure_qbob.build())

measure_op.simulate()

Preparing Q# environment...


[0, 0]